# 1. Data Import

In [ ]:
from utils_NB import *
import pandas as pd
import plotly.graph_objects as go

#OPTIMIZED_DATA_DIR = 'data/optimized'

df_pd  = pd.read_csv(f"{OPTIMIZED_DATA_DIR}/pd_solution.csv")
df_soc = pd.read_csv(f"{OPTIMIZED_DATA_DIR}/soc_solution.csv")
df_fc  = pd.read_csv(f"{OPTIMIZED_DATA_DIR}/forecast_generation_and_prices.csv")
df_ev  = pd.read_csv(f"{OPTIMIZED_DATA_DIR}/ev_data.csv")
df_ev = df_ev.set_index("unique_ev")
df_fc["timestamp"] = pd.to_datetime(df_fc["timestamp"])
df_ev["tod"] = pd.to_datetime(df_ev["tod"])
timestamp_to_t = {ts: idx for idx, ts in enumerate(df_fc["timestamp"])}

In [91]:
import pandas as pd
import pprint

summary = {}

# 1. Total EVs per day
df_ev_tmp = df_ev.reset_index()
df_ev_tmp["date"] = pd.to_datetime(df_ev_tmp["date"]).dt.date
evs_per_day = df_ev_tmp.groupby("date")["unique_ev"].nunique()

summary["Total EVs per day"] = evs_per_day.to_dict()

# 2. Average duration of stay (hours)
df_ev_tmp["toa"] = pd.to_datetime(df_ev_tmp["toa"])
df_ev_tmp["tod"] = pd.to_datetime(df_ev_tmp["tod"])
df_ev_tmp["duration_hours"] = (df_ev_tmp["tod"] - df_ev_tmp["toa"]).dt.total_seconds() / 3600
avg_duration = df_ev_tmp["duration_hours"].mean()

summary["Average duration of stay (hours)"] = round(avg_duration, 2)

# 3. Average battery capacity (kWh)
avg_capacity = df_ev_tmp["max_battery_capacity"].mean()
summary["Average battery capacity (kWh)"] = round(avg_capacity, 2)

# ----------------------------------------------
# Print a clean summary
# ----------------------------------------------
print("=== DATA SUMMARY ===")
print("EVs per day:")
for day, count in evs_per_day.items():
    print(f"  {day.strftime('%Y-%m-%d')}: {count}")
print(f"Average stay duration: {summary['Average duration of stay (hours)']} hours")
print(f"Average battery capacity: {summary['Average battery capacity (kWh)']} kWh")


=== DATA SUMMARY ===
EVs per day:
  2024-12-24: 568
  2024-12-25: 569
  2024-12-26: 578
  2024-12-27: 569
  2024-12-28: 496
  2024-12-29: 332
  2024-12-30: 221
Average stay duration: 14.97 hours
Average battery capacity: 68.64 kWh


# 2. Optimization Validation

## 2.1 PD violations check

In [50]:
print("=== VALIDATION: Power Draw Limits ===")


violations = []

# max charger power (adjust if needed)
MP = 11  

for ev, grp in df_pd.groupby("ev"):
    pd_max = grp["PD_kW"].max()
    if pd_max > MP + 1e-6:
        violations.append((ev, pd_max))

if not violations:
    print("OK: No EV exceeded its maximum charging power.")
else:
    print("WARNING: These EVs went over MP:")
    for ev, v in violations:
        print(f"{ev}: PD={v:.3f} kW")


=== VALIDATION: Power Draw Limits ===
OK: No EV exceeded its maximum charging power.


## 2.2 SOC violations check

### 2.2.1 EVs failed to meet their desired SOC

!!! This piece of code has high computation time owing to large amount of data operations.

In [56]:
records = []

for ev in df_ev.index:
    dep_ts = df_ev.loc[ev, "tod"].replace(minute=0, second=0)
    if dep_ts not in timestamp_to_t:
        continue

    t_d = timestamp_to_t[dep_ts]

    soc_row = df_soc[(df_soc["ev"] == ev) & (df_soc["t"] == t_d)]
    if soc_row.empty:
        continue

    soc_final = soc_row.iloc[0]["SOC_kWh"]

    cap = df_ev.loc[ev, "max_battery_capacity"]
    desired_pct = df_ev.loc[ev, "d_soc"]
    desired_kWh = desired_pct / 100 * cap

    # percent gap instead of kWh gap
    gap_pct = max(0, (desired_kWh - soc_final) / desired_kWh * 100)

    records.append({
        "ev": ev,
        "SOC_final_kWh": soc_final,
        "SOC_desired_kWh": desired_kWh,
        "gap_pct": gap_pct,      # percent shortfall
        "tardy": gap_pct > 1e-3  # tiny tolerance
    })

df_tardy = pd.DataFrame(records)

# Summary stats
total_ev = len(df_tardy)
num_tardy = df_tardy["tardy"].sum()
pct_tardy = 100 * num_tardy / total_ev if total_ev else 0

min_gap = df_tardy["gap_pct"].min()
mean_gap = df_tardy["gap_pct"].mean()
max_gap = df_tardy["gap_pct"].max()

print("=== TARDINESS SUMMARY ===")
print(f"Total EVs: {total_ev}")
print(f"Tardy count: {num_tardy}")
print(f"Tardy percent: {pct_tardy:.2f}%")
print("Gap stats (percent):")
print(f"  min gap: {min_gap:.2f}%")
print(f"  mean gap: {mean_gap:.2f}%")
print(f"  max gap: {max_gap:.2f}%")

=== TARDINESS SUMMARY ===
Total EVs: 3333
Tardy count: 59
Tardy percent: 1.77%
Gap stats (percent):
  min gap: 0.00%
  mean gap: 0.02%
  max gap: 2.98%


### 2.2.2 SOC crossing battery capacity checks

In [57]:
#df_ev = df_ev.set_index("unique_ev")

soc_cap_viol = []

for ev, grp in df_soc.groupby("ev"):
    soc_max = grp["SOC_kWh"].max()
    mc = df_ev.loc[ev, "max_battery_capacity"]

    if soc_max > mc + 1e-6:
        soc_cap_viol.append((ev, soc_max, mc))

if not soc_cap_viol:
    print("OK: No EV exceeded battery capacity.")
else:
    print("WARNING: SOC above capacity for:")
    for ev, s, mc in soc_cap_viol:
        print(f"{ev}: SOC={s:.2f}, MC={mc}")

OK: No EV exceeded battery capacity.


## 2.3 SOC violation visualizations

### 2.3.1 SOC Gap: Desired - Final SOC of EV

In [58]:
df_tardy_only = df_tardy[df_tardy["tardy"] == True]

fig_gap = px.bar(
    df_tardy_only,
    x="ev",
    y="gap_pct",
    title="SOC Gap (%) for EVs That Missed Their Target",
    labels={"gap_pct": "Gap (%)", "ev": "EV"},
    color="gap_pct",
)
fig_gap.update_layout(showlegend=False)
fig_gap.show()

### 2.3.2 SOC Gap Box Plot

In [59]:
fig_box = px.box(
    df_tardy_only,
    x="gap_pct",           # horizontal box
    orientation="h",
    title="SOC Gap Distribution (%) — Tardy EVs Only",
    points="all",
    labels={"gap_pct": "Gap (%)"},
)
fig_box.show()

# 3. Data Visualization

## 3.1 Charging (PD-SOC) timeline of an EV

### 3.1.1 Single Day

In [82]:
def plot_ev_profile_by_day(ev_id, date_str, df_pd, df_soc, df_fc):
    """
    ev_id: 'V1'
    date_str: '2024-12-24'
    """

    # build unique key used in df_pd and df_soc
    unique_ev = f"{ev_id}_{date_str}"

    # Filter records for this EV
    ev_pd  = df_pd[df_pd["ev"] == unique_ev].copy()
    ev_soc = df_soc[df_soc["ev"] == unique_ev].copy()

    if ev_pd.empty or ev_soc.empty:
        print(f"No data found for EV {unique_ev}")
        return
    
    # arrival & departure
    toa = df_ev.loc[unique_ev, "toa_hr"]
    tod = df_ev.loc[unique_ev, "tod_hr"]

    # max-cap + desired SOC
    cap = df_ev.loc[unique_ev, "max_battery_capacity"]
    desired_pct = df_ev.loc[unique_ev, "d_soc"]
    desired_kWh = desired_pct / 100 * cap

    # Merge timestamps
    df_fc_idx = df_fc.reset_index().rename(columns={"index": "t"})
    ev_pd  = ev_pd.merge(df_fc_idx[["t", "timestamp"]], on="t", how="left")
    ev_soc = ev_soc.merge(df_fc_idx[["t", "timestamp"]], on="t", how="left")

    # Filter just that day
    day = pd.to_datetime(date_str).date()
    ev_pd_day  = ev_pd[ev_pd["timestamp"].dt.date == day]
    ev_soc_day = ev_soc[ev_soc["timestamp"].dt.date == day]

    if ev_pd_day.empty or ev_soc_day.empty:
        print(f"No data for {unique_ev} on {date_str}")
        return

    fig = go.Figure()

    # PD trace
    fig.add_trace(go.Scatter(
        x=ev_pd_day["timestamp"],
        y=ev_pd_day["PD_kW"],
        name="PD (kW)",
        mode="lines",
        yaxis="y1"
    ))

    # SOC trace
    fig.add_trace(go.Scatter(
        x=ev_soc_day["timestamp"],
        y=ev_soc_day["SOC_kWh"],
        name="SOC (kWh)",
        mode="lines",
        yaxis="y2"
    ))

    # Vertical lines for arrival + departure
    fig.add_shape(
        type="line",
        x0=toa, x1=toa,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="green", width=2, dash="dot")
    )
    fig.add_shape(
        type="line",
        x0=tod, x1=tod,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="red", width=2, dash="dot")
    )

    # Horizontal capacity + desired SOC lines (SOC axis)
    fig.add_shape(
        type="line",
        x0=ev_soc_day["timestamp"].min(),
        x1=ev_soc_day["timestamp"].max(),
        y0=cap, y1=cap,
        xref="x", yref="y2",
        line=dict(color="black", width=2, dash="dash")
    )
    fig.add_shape(
        type="line",
        x0=ev_soc_day["timestamp"].min(),
        x1=ev_soc_day["timestamp"].max(),
        y0=desired_kWh, y1=desired_kWh,
        xref="x", yref="y2",
        line=dict(color="blue", width=2, dash="dash")
    )

    fig.update_layout(
        title=f"PD and SOC for {ev_id} on {date_str}",
        xaxis=dict(title="Time"),
        yaxis=dict(title="PD (kW)", side="right"),
        yaxis2=dict(
            title="SOC (kWh)",
            overlaying="y",
            side="left"
        ),
        template="plotly_white"
    )

    fig.show()


ev_id = 'V2'
date_val = '2024-12-26'
plot_ev_profile_by_day(ev_id, date_val, df_pd, df_soc, df_fc)

### 3.1.2 Complete Optimization Horizon along with Arrivals and Departures

In [83]:
df_ev_all = df_ev.copy()

# Ensure datetime columns behave like timestamps
df_ev_all["toa"] = pd.to_datetime(df_ev_all["toa"])
df_ev_all["tod"] = pd.to_datetime(df_ev_all["tod"])

# Extract hour-level arrival and departure
df_ev_all["arrival_hour"]   = df_ev_all["toa"].dt.floor("h")
df_ev_all["departure_hour"] = df_ev_all["tod"].dt.floor("h")

# Count arrivals and departures per hour
arrivals   = df_ev_all.groupby("arrival_hour").size().rename("arrivals")
departures = df_ev_all.groupby("departure_hour").size().rename("departures")

# Base timeline from forecast data
df_full = df_fc[["timestamp"]].copy()

# Merge arrival/departure counts into the timeline
df_full = df_full.merge(arrivals,   left_on="timestamp", right_index=True, how="left")
df_full = df_full.merge(departures, left_on="timestamp", right_index=True, how="left")
df_full = df_full.fillna(0)

# Add aggregated PD from df_pd long format
pd_tot = df_pd.groupby("t")["PD_kW"].sum().reindex(df_full.index, fill_value=0)
df_full["PD_total_kW"] = pd_tot.values

# -------------------------
# Build plot
# -------------------------
fig = go.Figure()

# Arrivals bar
fig.add_trace(go.Bar(
    x=df_full["timestamp"],
    y=df_full["arrivals"],
    name="Arrivals",
    marker_color="blue"
))

# Departures bar
fig.add_trace(go.Bar(
    x=df_full["timestamp"],
    y=df_full["departures"],
    name="Departures",
    marker_color="red"
))

# EV charging power on secondary axis
fig.add_trace(go.Scatter(
    x=df_full["timestamp"],
    y=df_full["PD_total_kW"],
    mode="lines",
    name="EV Charging (kW)",
    yaxis="y2",
    line=dict(color="black", width=3)
))

# -------------------------
# Layout
# -------------------------
fig.update_layout(
    title="Hourly EV Arrivals & Departures + Charging Power",
    barmode="stack",

    xaxis=dict(title="Time"),

    yaxis=dict(
        title="EV Count",
        side="left"
    ),

    yaxis2=dict(
        title="Charging Power (kW)",
        overlaying="y",
        side="right",
        showgrid=False
    ),

    template="plotly_white",
    legend=dict(title="Legend"),
    height=450
)

fig.show()

## 3.2 DA Generation + EV charging (Aggregated Power Draws from the grid) + DA Price

### 3.2.1 Single Day

In [80]:
# Pick a date
day_to_plot = "2024-12-24"
day_obj = pd.to_datetime(day_to_plot).date()

# 1. Filter forecast dataframe for that day
df_day = df_fc[df_fc["timestamp"].dt.date == day_obj].copy()

if df_day.empty:
    print(f"No data available for {day_to_plot}")
    raise SystemExit

# Assign hour index t (same as df_fc index)
df_day = df_day.reset_index().rename(columns={"index": "t"})

# 2. Compute aggregated EV PD (kW) from df_pd long format
pd_tot = (
    df_pd.groupby("t")["PD_kW"]
    .sum()
    .reindex(df_day["t"], fill_value=0)
)

df_day["PD_total_kW"] = pd_tot.values

# --------------------------------
# Build Plotly figure
# --------------------------------
fig = go.Figure()

# Stacked gen bars
gen_cols = [
    ("solar_generation",            "Solar (MW)"),
    ("wind_on_generation",          "Wind Onshore (MW)"),
    ("wind_off_generation",         "Wind Offshore (MW)"),
    ("fossil_hard_coal_generation", "Coal (MW)"),
    ("fossil_gas_generation",       "Gas (MW)")
]

for col, label in gen_cols:
    fig.add_trace(go.Bar(
        x=df_day["timestamp"],
        y=df_day[col],
        name=label,
        opacity=0.8
    ))

# EV charging (kW) — secondary axis
fig.add_trace(go.Scatter(
    x=df_day["timestamp"],
    y=df_day["PD_total_kW"],
    name="EV Charging (kW)",
    mode="lines+markers",
    yaxis="y2",
    line=dict(color="black", width=3),
    marker=dict(color="black")
))

# Price (EUR/MWh) — third axis
fig.add_trace(go.Scatter(
    x=df_day["timestamp"],
    y=df_day["forecasted_prices"],
    name="DA Price (EUR/MWh)",
    mode="lines",
    yaxis="y3",
    line=dict(color="blue", width=3, dash="dot")
))

# --------------------------------
# Layout
# --------------------------------
fig.update_layout(
    title=f"Generation Mix, EV Charging, and Day-Ahead Price on {day_to_plot}",

    xaxis=dict(title="Time", domain=[0.1, 1]),

    # Left axis: MW
    yaxis=dict(
        title="Generation (MW)",
        side="left",
        anchor="x",
        position=0
    ),

    # Right axis (inside): EV charging (kW)
    yaxis2=dict(
        title="EV Charging (kW)",
        overlaying="y",
        side="right",
        position=1,
        showgrid=False
    ),

    # Third axis (left, slightly shifted): price
    yaxis3=dict(
        title="DA Price (EUR/MWh)",
        overlaying="y",
        side="left",
        anchor="free",
        position=0.05,
        showgrid=False
    ),

    barmode="stack",
    template="plotly_white",
    height=500,
    margin=dict(r=120),

    legend=dict(
        title="Legend",
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=0.5
    )
)

fig.show()

### 3.2.2 Complete Optimization Horizon

In [85]:
df_full = df_fc.copy().reset_index(drop=True)

# Compute total PD per hour
pd_total = df_pd.groupby("t")["PD_kW"].sum().reindex(df_full.index, fill_value=0)
df_full["PD_total_kW"] = pd_total.values

# -------------------------------------
# Build figure
# -------------------------------------
fig = go.Figure()

# 1. Generation mix as stacked area
gen_cols = [
    ('solar_generation', 'Solar'),
    ('wind_on_generation', 'Wind Onshore'),
    ('wind_off_generation', 'Wind Offshore'),
    ('fossil_hard_coal_generation', 'Coal'),
    ('fossil_gas_generation', 'Gas'),
]

for col, label in gen_cols:
    fig.add_trace(go.Scatter(
        x=df_full["timestamp"],
        y=df_full[col],
        name=label,
        mode="lines",
        stackgroup='gen',     # this stacks areas
        line=dict(width=0.5),
        opacity=0.8
    ))

# 2. EV charging line (kW) — y2 axis
fig.add_trace(go.Scatter(
    x=df_full["timestamp"],
    y=df_full["PD_total_kW"],
    mode="lines",
    name="EV Charging (kW)",
    yaxis="y2",
    line=dict(color="black", width=3)
))

# 3. Day-ahead prices — y3 axis
fig.add_trace(go.Scatter(
    x=df_full["timestamp"],
    y=df_full["forecasted_prices"],
    mode="lines",
    name="DA Price (EUR/MWh)",
    yaxis="y3",
    line=dict(color="blue", width=4, dash="dot")
))

# -------------------------------------
# Layout
# -------------------------------------
fig.update_layout(
    title="Generation Mix, EV Charging, and Day-Ahead Price",
    xaxis=dict(
        title="Time",
        domain=[0.1, 1]
    ),

    yaxis=dict(
        title="Generation (MW)",
        side="left",
        anchor="x",
        position=0
    ),

    yaxis2=dict(
        title="EV Charging (kW)",
        overlaying="y",
        side="right",
        position=1,
        showgrid=False
    ),

    yaxis3=dict(
        title="DA Price (EUR/MWh)",
        overlaying="y",
        side="left",
        anchor="free",
        position=0.05,
        showgrid=True
    ),

    template="plotly_white",
    margin=dict(r=120),
    height=550,

    legend=dict(
        title="Legend",
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=0.5
    )
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Pick a date
day_to_plot = "2024-12-24"

# Extract that day's rows from forecast dataframe
df_day = df_fc[df_fc['timestamp'].dt.date.astype(str) == day_to_plot].copy()

# Compute aggregated EV PD for each hour in this day (using EV-specific indices)
pd_tot = []
for idx, row in df_day.iterrows():
    t = time_to_idx[row['timestamp']]
    total_pd = sum(PD_solution.get((ev, t), 0.0) for ev in model.EV)  # kW
    pd_tot.append(total_pd)

df_day["PD_total_kW"] = pd_tot

# ---------------------------
# Build Plotly Figure
# ---------------------------
fig = go.Figure()

# Stacked generation bars (MW)
fig.add_trace(go.Bar(
    x=df_day["timestamp"],
    y=df_day["solar_generation"],
    name="Solar (MW)"
))
fig.add_trace(go.Bar(
    x=df_day["timestamp"],
    y=df_day["wind_on_generation"],
    name="Wind Onshore (MW)"
))
fig.add_trace(go.Bar(
    x=df_day["timestamp"],
    y=df_day["wind_off_generation"],
    name="Wind Offshore (MW)"
))
fig.add_trace(go.Bar(
    x=df_day["timestamp"],
    y=df_day["fossil_hard_coal_generation"],
    name="Coal (MW)"
))
fig.add_trace(go.Bar(
    x=df_day["timestamp"],
    y=df_day["fossil_gas_generation"],
    name="Gas (MW)"
))

# EV charging line on secondary axis (kW)
fig.add_trace(go.Scatter(
    x=df_day["timestamp"],
    y=df_day["PD_total_kW"],
    mode="lines+markers",
    name="EV Charging (kW)",
    yaxis="y2",
    line=dict(color="black", width=3),
    marker=dict(color="black")
))

fig.update_layout(
    title=f"Generation Mix (MW) and EV Charging (kW) on {day_to_plot}",
    xaxis=dict(title="Time"),
    yaxis=dict(title="Generation (MW)"),
    yaxis2=dict(
        title="EV Charging (kW)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    barmode="stack",
    template="plotly_white",
    legend=dict(title="Legend")
)

fig.show()
